In [2]:
#Fetch Gemini API Key
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Gemini API Key:")

**bbc** - 크롤링 불가, 유기 <br>
**블랙월** - 광고 너무 많아서 유기, 요약문만 사용? <br>
**뉴욕타임즈** - API사용, 요약본, URL, 타이틀<br>
**CNN** - 전문 긁어오기

In [48]:
import numpy
import pandas as pd

# 가사 데이터 수집

In [ ]:
# cnn 기사 데이터 수집
# cnn_news.csv 파일 경로 수정
cnn=pd.read_csv("../data/cnn_news.csv")

# 기사 선정 기준? 
import random
while True: 
    i=int(random.random()*100)
    article=cnn.iloc[i]
    if not (article.content=='' or article.content=="No content available."): break
article

title      Italy releases Iranian businessman wanted by U...
link       https://edition.cnn.com/2025/01/12/europe/ital...
content    Iranian businessman Mohammad Abedini, who was ...
Name: 4, dtype: object

In [88]:
print(article.content)

Iranian businessman Mohammad Abedini, who was detained in Italy at the request of the United States following a drone attack that killed three Americans, has been released. Abedini, 38, was arrested i...


# full code

In [94]:
# CNN full code
import time
import google.generativeai as genai


# model for CNN, summarize
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
#system_instruction=
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article.content}])

# create summary
start=time.time()
summary = article.title
summary += chat.send_message("입력된 내용을 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해.").text
summary += chat.send_message("요약본을 한국어로 번역해줘").text
end=time.time()
print(f"summary created: {end - start:.5f}sec")

# create vocab
start=time.time()
prompt="""
영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해. 각 단어 사이에는 개행해. 
예시: 
collates 모으다
Crucial 중요한
Attributed 귀속된
"""

response = chat.send_message(prompt)
response=response.text.split('\n')
response.pop()
vocab={}
for i in range(len(response)):
    v,m=response[i].split(maxsplit=1)
    vocab[v]=m
end=time.time()
print(f"vocab created: {end - start:.5f}sec")

summary created: 2.72702sec
vocab created: 0.49353sec


# Summary

In [ ]:
#summary 생성 셀
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
#system_instruction=
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article}])

# create summary
start=time.time()
summary = chat.send_message("가지고 온 영어로된 기사를 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해.")
summary += chat.send_message("요약본을 한국어로 번역해줘")
end=time.time()
print(f"summary created: {end - start:.5f}sec")

In [95]:
print(summary)

Italy releases Iranian businessman wanted by US over drone attack that killed AmericansIranian businessman Mohammad Abedini, 38, was detained in Italy at the US's request following a drone strike that killed three Americans.  He has since been released.  The reason for his detention and the specifics of the drone strike remain unclear.
38세 이란 사업가 모하마드 아베디니가 미국인 3명이 사망한 드론 공격 이후 미국의 요청으로 이탈리아에서 체포되었으나 석방되었습니다.  체포 이유와 드론 공격의 구체적인 내용은 아직 불분명합니다.



In [89]:
print("원문 길이:", len(article.content))
print("요약문 길이:", len(summary.split("\n")[0]))

원문 길이: 203
요약문 길이: 339


In [97]:
# 영어 요약문
summary.split("\n")[0]

# 한국어 요약문
summary.split("\n")[1]

'38세 이란 사업가 모하마드 아베디니가 미국인 3명이 사망한 드론 공격 이후 미국의 요청으로 이탈리아에서 체포되었으나 석방되었습니다.  체포 이유와 드론 공격의 구체적인 내용은 아직 불분명합니다.'

# 단어 추출(vocab)

In [45]:
# 단어 추출
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article}])

# create vocab
start=time.time()
prompt="""
영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해. 각 단어 사이에는 개행해. 
예시: 
collates 모으다
Crucial 중요한
Attributed 귀속된
"""

response = chat.send_message(prompt)
response=response.text.split('\n')
response.pop()
vocab={}
for i in range(len(response)):
    v,m=response[i].split(maxsplit=1)
    vocab[v]=m
end=time.time()
print(f"vocab created: {end - start:.5f}sec")

vocab created: 0.54811sec


In [91]:
vocab

{'detained': '체포된', 'request': '요청', 'following': '~이후에'}

In [69]:
chat.history

[parts {
   text: "A former child star from Australia died when the Los Angeles wildfires ripped through his family’s Malibu estate in California earlier this week, according to his mother. Rory Callum Sykes was at the ..."
 }
 role: "user",
 parts {
   text: "가지고 온 영어로된 기사를 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해."
 }
 role: "user",
 parts {
   text: "Please provide me with the English article you\'d like me to summarize.  I need the text of the article to create a 100-word summary.\n"
 }
 role: "model",
 parts {
   text: "요약본을 한국어로 번역해줘"
 }
 role: "user",
 parts {
   text: "죄송합니다. 제게 기사 내용이 없어서 요약본을 한국어로 번역할 수 없습니다. 기사를 제공해주시면 제가 요약하고 한국어로 번역해 드리겠습니다.\n"
 }
 role: "model",
 parts {
   text: "\n영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해.\n예시: \ncollates 모으다\nCrucial 중요한\nAttributed 귀속된\n"
 }
 role: "user",
 parts {
   text: "estate 재산\nwildfires 산불\nreportedly 보도에 따르면\n"
 }
 role: "model"]